In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 60.1 MB/s eta 0:00:00


In [ ]:
!sudo apt-get remove libav-tools

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package 'libav-tools' is not installed, so not removed
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libasound2-dev is already the newest version (1.2.6.1-1ubuntu1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (210 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 123632 files and directories currently installed.)
Pre

In [ ]:
!pip install pyaudio soundfile moviepy pydub gtts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=PyAudio-0.2.14-cp310-cp310-linux_x86_64.whl size=63858 sha256=34673a8ff741073faa5d77c5d85f85bf5fc704abb23e54e05bf95dd1fcf4140f
  Stored in directory: /root/.cache/pip/wheels/d6/21/f4/0b51d41ba79e51b16295cbb096ec49f334792814d545b508c5
Successfully built pyaudio


In [ ]:
from IPython.display import Javascript, display, Audio
from google.colab import output
from base64 import b64decode
import speech_recognition as sr
from io import BytesIO
from pydub import AudioSegment
import torch
from transformers import AutoProcessor, AutoModelForPreTraining
from PIL import Image
import pickle as pkl
import re
from gtts import gTTS
import ipywidgets as widgets
import time

In [ ]:
#You can download this trained model and processor file from here
#model ==> https://drive.google.com/uc?id=1HD-pE25wsGhbKylGKEKv3TBZadia_4dy&export=download
#processor ==> https://drive.google.com/uc?id=1-303tkfqd32yT_01MxTtei7sF-VnMAFh&export=download

model = pkl.load(open("/content/drive/MyDrive/Hemanth/VQA/model/humanvlm_model.pkl", "rb"))
processor = pkl.load(open("/content/drive/MyDrive/Hemanth/VQA/model/humanvlm_processor.pkl", "rb"))

In [ ]:
# JavaScript for audio recording
RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time));
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader();
  reader.onloadend = e => resolve(e.srcElement.result);
  reader.readAsDataURL(blob);
});
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  recorder = new MediaRecorder(stream, { mimeType: 'audio/webm' });
  chunks = [];
  recorder.ondataavailable = e => chunks.push(e.data);
  recorder.start();
  await sleep(time);
  recorder.onstop = async () => {
    blob = new Blob(chunks);
    text = await b2text(blob);
    resolve(text);
  };
  recorder.stop();
});
"""

# Function to record speech
def record(sec=3):
    display(Javascript(RECORD))
    s = output.eval_js('record(%d)' % (sec * 1000))
    b = b64decode(s.split(',')[1])
    audio = AudioSegment.from_file(BytesIO(b), format="webm")
    return audio.export("temp.wav", format="wav")

def recognize_speech():
    print("Speak Now...")
    record(3)  # Record for 5 seconds
    recognizer = sr.Recognizer()
    with sr.AudioFile("temp.wav") as source:
        audio = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio)
        print(f"You said: {text}")
        return text
    except sr.UnknownValueError:
        print("Could not understand the audio.")
        return ""
    except sr.RequestError as e:
        print(f"Request error: {e}")
        return ""


In [ ]:
# Capture image from webcam
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();
      await new Promise(resolve => capture.onclick = resolve);
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getTracks().forEach(track => track.stop());
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = output.eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

In [ ]:
# VQA function
def perform_vqa(image_path, question):
    # Open the image using PIL
    image = Image.open(image_path).convert("RGB")

    prompt_style = "Explain in a simple and less than 10 words."
    prompt = f"USER: <image>\n{question}\n{prompt_style}\nASSISTANT:"
    inputs = processor(images=image, text=prompt, return_tensors="pt").to("cuda")

    for key in inputs:
        if inputs[key].dtype == torch.float32:  # Check if the tensor is a floating-point type
            inputs[key] = inputs[key].type(torch.cuda.HalfTensor)

    output = model.generate(**inputs, max_new_tokens=15, do_sample=False)
    result = processor.decode(output[0], skip_special_tokens=True)

    assistant_response = result.split("ASSISTANT:")[1].strip()
    first_sentence = re.split(r'[.!?]', assistant_response)[0]  # Split by sentence-ending punctuation
    cleaned_response = first_sentence.strip() + "."
    return cleaned_response


In [ ]:
# Convert text to speech and play
def text_to_speech(text):
    tts = gTTS(text=text, lang='en')
    tts.save("output.mp3")
    return Audio("output.mp3", autoplay=True)

# Main interaction loop
button = widgets.Button(description="Speak")

def on_button_clicked(b):
    question = recognize_speech()
    if question:
        photo_path = take_photo()
        answer = perform_vqa(photo_path, question)
        print(f"Assistant's Answer: {answer}")
        display(text_to_speech(answer))

In [ ]:
button.on_click(on_button_clicked)
display(button)

Button(description='Speak', style=ButtonStyle())

Speak Now...


<IPython.core.display.Javascript object>

You said: does


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

You said: what's that symbol


<IPython.core.display.Javascript object>

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Assistant's Answer: The symbol is a bull's head.
